# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [3]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
from pathlib import Path
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dredm\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# Read your api key environment variable
load_dotenv('C:/Users/dredm/Desktop/Daryl/Fintech_Bootcamp/Git_Repos/uw-fintech-spring-21/.env')
news_api_key = os.getenv("NEWS_API_KEY")

type(news_api_key)

str

In [21]:
#os.getcwd()

In [5]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=news_api_key)

In [6]:
# Fetch the Bitcoin news articles
bitcoin_news = newsapi.get_everything(
    q="bitcoin",
    language="en"
)
bitcoin_news["totalResults"]


10213

In [7]:
# Fetch the Ethereum news articles
ethereum_news = newsapi.get_everything(
    q="ethereum",
    language="en"
)
ethereum_news["totalResults"]

4065

In [8]:
#Create DataFrame function
def create_df(news, language):
    articles=[]
    for article in news:
        try:
            title = article["title"],
            description = article["description"],
            text = article["content"],
            date = article["publishedAt"][:10]
            
            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass
    return pd.DataFrame(articles)



In [9]:
# Create the Bitcoin Dataframe and save it as csv with encoding utf-8-sig
bitcoin_df = create_df(bitcoin_news["articles"], "en")
#bitcoin_df.head()
file_path = Path("bitcoin_df_encoded.csv")
bitcoin_df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [10]:
#Load the encoded Bitcoin CSV file
file_path = Path("bitcoin_df_encoded.csv")
bitcoin_enc_df = pd.read_csv(file_path, encoding="utf-8-sig")
bitcoin_enc_df.head()

,title,description,text,date,language
0,('What Square’s smashing earnings tell us abou...,"(""Today we're talking Square earnings and its ...",('Shares of Square are up more than 6% today a...,2021-05-07,en
1,('Tesla sees bitcoin as important financial to...,('Tesla’s relationship with bitcoin is not a d...,('Tesla’s relationship with bitcoin is not a d...,2021-04-27,en
2,"(""You Can Now Exchange Crypto on Venmo, If You...",('Cryptocurrency continues to gain mainstream ...,('Cryptocurrency continues to gain mainstream ...,2021-04-20,en
3,('African crypto usage spurs Luno as customers...,('The crypto industry as a whole has seen a mo...,('The crypto industry as a whole has seen a mo...,2021-04-13,en
4,('Crypto market takes a dive with Bitcoin lead...,('Cryptocurrency prices continued to tumble Fr...,('Cryptocurrency prices continued to tumble Fr...,2021-04-23,en


In [11]:
# Create the Bitcoin sentiment scores DataFrame

bitcoin_title_sent = {
    "title_compound": [],
    "title_pos": [],
    "title_neu": [],
    "title_neg": [],
    #"title_sent": [],
}

bitcoin_text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    #"text_sent": [],
    
}

for index, row in bitcoin_enc_df.iterrows():
    try:
        title_sentiment = analyzer.polarity_scores(row["title"])
        bitcoin_title_sent["title_compound"].append(title_sentiment["compound"])
        bitcoin_title_sent["title_pos"].append(title_sentiment["pos"])
        bitcoin_title_sent["title_neu"].append(title_sentiment["neu"])
        bitcoin_title_sent["title_neg"].append(title_sentiment["neg"])
        #bitcoin_title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))
        
        text_sentiment = analyzer.polarity_scores(row["text"])
        bitcoin_text_sent["text_compound"].append(text_sentiment["compound"])
        bitcoin_text_sent["text_pos"].append(text_sentiment["pos"])
        bitcoin_text_sent["text_neu"].append(text_sentiment["neu"])
        bitcoin_text_sent["text_neg"].append(text_sentiment["neg"])
        #bitcoin_text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

bitcoin_title_sentiment_df = pd.DataFrame(bitcoin_title_sent)    
bitcoin_text_sentiment_df = pd.DataFrame(bitcoin_text_sent)
bitcoin_sentiment_df = bitcoin_title_sentiment_df.join(bitcoin_text_sentiment_df)
#display(bitcoin_title_sentiment_df)

#display(bitcoin_text_sentiment_df)
bitcoin_sentiment_df.head()

,title_compound,title_pos,title_neu,title_neg,text_compound,text_pos,text_neu,text_neg
0,0.0000,0.000,1.000,0.0,0.0000,0.000,1.000,0.0
1,0.2023,0.153,0.847,0.0,0.0000,0.000,1.000,0.0
2,0.0000,0.000,1.000,0.0,0.7506,0.171,0.829,0.0
3,0.0258,0.121,0.879,0.0,0.6124,0.135,0.865,0.0
4,0.0000,0.000,1.000,0.0,0.0000,0.000,1.000,0.0


In [12]:
# Create the Ethereum Dataframe and save it as csv with encoding utf-8-sig
ethereum_df = create_df(ethereum_news["articles"], "en")
#bitcoin_df.head()
file_path = Path("ethereum_df_encoded.csv")
ethereum_df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [13]:
#Load the encoded Bitcoin CSV file
file_path = Path("ethereum_df_encoded.csv")
ethereum_enc_df = pd.read_csv(file_path, encoding="utf-8-sig")
ethereum_enc_df.head()

,title,description,text,date,language
0,('Radical Ethereum entrepreneurs are redefinin...,('Their investors call them disruptive innovat...,('Their investors call them disruptive innovat...,2021-05-04,en
1,"(""You Can Now Exchange Crypto on Venmo, If You...",('Cryptocurrency continues to gain mainstream ...,('Cryptocurrency continues to gain mainstream ...,2021-04-20,en
2,('Venmo now lets users buy and sell Bitcoin an...,('Venmo is jumping aboard the cryptocurrency b...,('Venmo is jumping aboard the cryptocurrency b...,2021-04-20,en
3,"('Move over, Bitcoin. Ethereum is at an all-ti...","(""Bitcoin prices continued their rebound Satur...","(""New York (CNN Business)Bitcoin prices contin...",2021-05-01,en
4,('CryptoPunks maker Larva Labs launches their ...,"('The creators behind CryptoPunks, one of the ...","('The creators behind CryptoPunks, one of the ...",2021-05-03,en


In [14]:
# Create the Ethereum sentiment scores DataFrame

ethereum_title_sent = {
    "title_compound": [],
    "title_pos": [],
    "title_neu": [],
    "title_neg": [],
    }

ethereum_text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
        
}

for index, row in ethereum_enc_df.iterrows():
    try:
        title_sentiment = analyzer.polarity_scores(row["title"])
        ethereum_title_sent["title_compound"].append(title_sentiment["compound"])
        ethereum_title_sent["title_pos"].append(title_sentiment["pos"])
        ethereum_title_sent["title_neu"].append(title_sentiment["neu"])
        ethereum_title_sent["title_neg"].append(title_sentiment["neg"])
                
        text_sentiment = analyzer.polarity_scores(row["text"])
        ethereum_text_sent["text_compound"].append(text_sentiment["compound"])
        ethereum_text_sent["text_pos"].append(text_sentiment["pos"])
        ethereum_text_sent["text_neu"].append(text_sentiment["neu"])
        ethereum_text_sent["text_neg"].append(text_sentiment["neg"])
        
    except AttributeError:
        pass

ethereum_title_sentiment_df = pd.DataFrame(ethereum_title_sent)    
ethereum_text_sentiment_df = pd.DataFrame(ethereum_text_sent)
ethereum_sentiment_df = ethereum_title_sentiment_df.join(ethereum_text_sentiment_df)
#display(ethereum_title_sentiment_df)

#display(bitcoin_text_sentiment_df)
ethereum_sentiment_df.head()

,title_compound,title_pos,title_neu,title_neg,text_compound,text_pos,text_neu,text_neg
0,0.0,0.0,1.0,0.0,-0.4019,0.072,0.778,0.15
1,0.0,0.0,1.0,0.0,0.7506,0.171,0.829,0.00
2,0.0,0.0,1.0,0.0,0.0258,0.037,0.963,0.00
3,0.0,0.0,1.0,0.0,0.0000,0.000,1.000,0.00
4,0.0,0.0,1.0,0.0,0.4754,0.091,0.909,0.00


In [15]:
# Describe the Bitcoin Sentiment
bitcoin_sentiment_df.describe()

,title_compound,title_pos,title_neu,title_neg,text_compound,text_pos,text_neu,text_neg
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000
mean,0.064390,0.067800,0.906100,0.026100,0.162365,0.049150,0.94035,0.010550
std,0.245910,0.129863,0.132885,0.067227,0.297048,0.068475,0.07797,0.026112
min,-0.401900,0.000000,0.490000,0.000000,-0.381800,0.000000,0.77700,0.000000
25%,0.000000,0.000000,0.839000,0.000000,0.000000,0.000000,0.89575,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.00000,0.000000
75%,0.069925,0.126500,1.000000,0.000000,0.360375,0.099000,1.00000,0.000000
max,0.636900,0.510000,1.000000,0.213000,0.750600,0.183000,1.00000,0.085000


In [16]:
# Describe the Ethereum Sentiment
ethereum_sentiment_df.describe()

,title_compound,title_pos,title_neu,title_neg,text_compound,text_pos,text_neu,text_neg
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.082220,0.094800,0.864400,0.040800,0.111150,0.042800,0.945900,0.011300
std,0.266191,0.167328,0.185735,0.087978,0.299861,0.059696,0.070965,0.036794
min,-0.391900,0.000000,0.492000,0.000000,-0.401900,0.000000,0.778000,0.000000
25%,0.000000,0.000000,0.744500,0.000000,0.000000,0.000000,0.923250,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.018500,0.962500,0.000000
75%,0.202300,0.193500,1.000000,0.000000,0.164300,0.065250,1.000000,0.000000
max,0.636900,0.508000,1.000000,0.275000,0.817600,0.210000,1.000000,0.150000


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum had the higher mean positive score when evaluating just the titles, but Bitcoin had the higher mean positive score when evaluating the text.

Q: Which coin had the highest compound score?

A: Both coins had the same max compound score for titles, but Ethereum had a higher max compound score for text.

Q. Which coin had the highest positive score?

A: Bitcoin had the highest positive score for titles, but Ethereum had the highest positive score for text.

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [17]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [18]:
# Instantiate the lemmatizer
lmt = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
sw_addons = {'said'}

In [19]:
# Complete the tokenizer function

def tokenizer(text):
    """Tokenizes text."""
    tokens = []
    
    for index, row in text.iterrows():
        # Remove the punctuation from text
        regex = re.compile("[^a-zA-Z]")
        re_clean = regex.sub(' ', row['text'])
        
        # Create a tokenized list of the words
        words = word_tokenize(re_clean)
        
        # Lemmatize words into root words
        lem = [lmt.lemmatize(word) for word in words]
        
        # Convert the words to lowercase
        lower_case = [word.lower() for word in lem]
        
        # Remove the stop words
        stp = [word for word in lower_case if word not in sw.union(sw_addons)]
        tokens.append(stp)
    
    return tokens


In [30]:
# Create a new tokens column for Bitcoin
#print(set(tokenizer(bitcoin_enc_df)))
bit_tokens = tokenizer(bitcoin_enc_df)
#bitcoin_enc_df['tokens'] = bit_tokens
#bitcoin_enc_df.head()
bit_tokens

[['shares',
  'square',
  'today',
  'american',
  'fintech',
  'company',
  'reported',
  'staggering',
  'billion',
  'revenue',
  'q',
  'earnings',
  'report',
  'far',
  'ahead',
  'expected',
  'tally',
  'bi',
  'char'],
 ['tesla',
  'relationship',
  'bitcoin',
  'dalliance',
  'according',
  'comment',
  'made',
  'company',
  'cfo',
  'dubbed',
  'master',
  'coin',
  'zach',
  'kirkhorn',
  'earnings',
  'call',
  'monday',
  'instead',
  'compan',
  'char'],
 ['cryptocurrency',
  'continues',
  'gain',
  'mainstream',
  'acceptance',
  'paypal',
  'announced',
  'added',
  'bitcoin',
  'ethereum',
  'bitcoin',
  'cash',
  'litecoin',
  'venmo',
  'app',
  'user',
  'base',
  'million',
  'mo',
  'char'],
 ['crypto',
  'industry',
  'whole',
  'ha',
  'seen',
  'momentous',
  'year',
  'growth',
  'heavily',
  'spurred',
  'entrance',
  'institutional',
  'investor',
  'adopting',
  'bitcoin',
  'due',
  'store',
  'value',
  'property',
  'spike',
  'char'],
 ['cryptocurren

In [21]:
# Create a new tokens column for Ethereum
eth_tokens = tokenizer(ethereum_enc_df)
ethereum_enc_df['tokens'] = eth_tokens
ethereum_enc_df.head()

,title,description,text,date,language,tokens
0,('Radical Ethereum entrepreneurs are redefinin...,('Their investors call them disruptive innovat...,('Their investors call them disruptive innovat...,2021-05-04,en,"[investor, call, disruptive, innovator, detrac..."
1,"(""You Can Now Exchange Crypto on Venmo, If You...",('Cryptocurrency continues to gain mainstream ...,('Cryptocurrency continues to gain mainstream ...,2021-04-20,en,"[cryptocurrency, continues, gain, mainstream, ..."
2,('Venmo now lets users buy and sell Bitcoin an...,('Venmo is jumping aboard the cryptocurrency b...,('Venmo is jumping aboard the cryptocurrency b...,2021-04-20,en,"[venmo, jumping, aboard, cryptocurrency, bandw..."
3,"('Move over, Bitcoin. Ethereum is at an all-ti...","(""Bitcoin prices continued their rebound Satur...","(""New York (CNN Business)Bitcoin prices contin...",2021-05-01,en,"[new, york, cnn, business, bitcoin, price, con..."
4,('CryptoPunks maker Larva Labs launches their ...,"('The creators behind CryptoPunks, one of the ...","('The creators behind CryptoPunks, one of the ...",2021-05-03,en,"[creator, behind, cryptopunks, one, popular, n..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [22]:
from collections import Counter
from nltk import ngrams

In [34]:
# Generate the Bitcoin N-grams where N=2
str1=' '
for item in bit_tokens[1]:
    str1+=item

#print(str1)
    
bigrams = ngrams(str1, n=2)
bigrams()

TypeError: 'generator' object is not callable

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---